In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
!pip install imageai
!pip install Pillow

In [3]:
import os
from pathlib import Path
cwd = Path(os.getcwd())
data_path = cwd / 'data_preparation/output/smash'

In [58]:
from PIL import Image, ImageOps

# assumes RGB
def transform_image(im):
    desired_size = 224

    old_size = im.size  # old_size[0] is in (width, height) format

    ratio = float(desired_size)/max(old_size)
    new_size = tuple([int(x*ratio) for x in old_size])
    # use thumbnail() or resize() method to resize the input image

    # thumbnail is a in-place operation

    # im.thumbnail(new_size, Image.ANTIALIAS)

    im = im.resize(new_size, Image.ANTIALIAS)
    # create a new image and paste the resized on it

    new_im = Image.new("RGB", (desired_size, desired_size))
    new_im.paste(im, ((desired_size-new_size[0])//2,
                        (desired_size-new_size[1])//2))
    return new_im

In [59]:
import glob2
import PIL.Image as Image

def split_fp(path):
    split_path = str(path).split('/')
    return '/'.join(split_path[:-1]), split_path[-1]

def split_fn(fn):
    split_fn = fn.split('.')
    return '.'.join(split_fn[:-1]), split_fn[-1]

def resize_images(image_dir, output_dir=None, prefix='', suffix=''):
    for fp in glob2.glob(str(image_dir)):
        im = Image.open(fp)
        im = transform_image(im)
        dir_, fn = split_fp(fp)
        base, ext = split_fn(fn)
        if output_dir:
            os.makedirs(output_dir, exist_ok=True)
            dir_ = output_dir
        im.save(Path(dir_) / (prefix + base + suffix + '.' + ext))

In [51]:
image_dir = data_path / 'test/melee_game/*.png'
resize_images(image_dir, output_dir='./test', suffix='.small')

In [ ]:
# https://imageai.readthedocs.io/en/latest/custom/
from imageai.Prediction.Custom import ModelTraining

model_trainer = ModelTraining()
model_trainer.setModelTypeAsDenseNet()
model_trainer.setDataDirectory(data_path)
model_trainer.trainModel(num_objects=2, num_experiments=100, enhance_data=True, batch_size=32, show_network_summary=True)

Model: "densenet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_173 (Conv2D)             (None, 112, 112, 64) 9408        input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_174 (BatchN (None, 112, 112, 64) 256         conv2d_173[0][0]                 
__________________________________________________________________________________________________
activation_171 (Activation)     (None, 112, 112, 64) 0           batch_normalization_174[0][0]    
___________________________________________________________________________________________